In [4]:
## Package Needed
import numpy as np
from scipy import signal
from matplotlib import pyplot as plt 
import scipy
from scipy.signal import savgol_filter

**INPUT:** 

**data:** 4 x Timeseries Location data. data_t: Thrust Location data; data_p: Pitch Location data; data_y: Yaw Location data.

**window_size:** I think 51 - 201 is a good number for this particular time seriesThe length of the filter window (i.e., the number of coefficients). window_length must be a positive odd integer. If mode is ‘interp’, window_length must be less than or equal to the size of x.

**factor_noisefloor:** The number of standard deviation you set for calculating the noise floor. I normally set it to 1


**OUTPUT:**

**input_t:** Thrust Input

**input_y:** Yaw Input

**input_p:** Pitch Input

In [5]:
def get_thrustinput(data_t,window_size,factor_noisefloor):

  diff_fs = np.diff(data_t[0]);
  # Step1: Getting speed for Thrust Player's movement 
  for i in range(len(diff_fs)):
    if diff_fs[i] < - 1000:
        diff_fs[i]=100
    elif diff_fs[i] > 500:
        diff_fs[i]=100
      
  # Step2: Filter speed data
  filt_fs = np.round(scipy.signal.savgol_filter(diff_fs, window_size, 2, mode='nearest'));
  
  # Step4: Get the accelearation
  acc_fs = np.diff(filt_fs)

  # Step5: calulate noise floor 
  nonzero_fs = np.nonzero(acc_fs)
  diff_fs_val = acc_fs[nonzero_fs]
  diff_fs_mean = np.mean(diff_fs_val)
  diff_fs_std = np.std(acc_fs[nonzero_fs])
    
  # Step6: get speed input
  input_t = np.zeros(len(diff_fs))
  for i in range(len(diff_fs)-1):
    if acc_fs[i] < -1* (diff_fs_mean+diff_fs_std*factor_noisefloor):
      input_t[i] = -1
    elif acc_fs[i] > (diff_fs_mean+diff_fs_std*factor_noisefloor):
      input_t[i] = 1
    else:
      input_t[i] = 0
  return input_t

In [6]:
def get_pitchinput(data_p,window_size,factor_noisefloor):

  # Step1 Pitch Data Cleaning 
  data_p[1]=data_p[1]-1000000
  for i in range(len(data_p[1])):
    if data_p[1][i] < -5000:
      data_p[1][i]=0

  # Step2 Apply Filters 
  f_lr = np.round(scipy.signal.savgol_filter(data_p[1], window_size, 2, mode='nearest'));

  # Step3 Take The Derivative
  diff_lr = np.diff(f_lr)

  # Step4: calulate noise floor 
  nonzero_lr = np.nonzero(diff_lr)
  diff_lr_val = diff_lr[nonzero_lr]
  diff_lr_mean = np.mean(diff_lr_val)
  diff_lr_std = np.std(diff_lr[nonzero_lr])

  # Step5: Retrieve Player Output
  input_p = np.zeros(len(diff_lr))
  for i in range(len(diff_lr)-1):
    if diff_lr[i] < -1*(diff_lr_mean+diff_lr_std*factor_noisefloor):
      input_p[i] = -1
    elif diff_lr[i] > (diff_lr_mean+diff_lr_std*factor_noisefloor):
      input_p[i] = 1
    else:
      input_p[i] = 0
    
  return input_p

  

In [7]:
def get_yawinput(data_y,window_size,factor_noisefloor):
  
  # Step1 Apply Filters to Yaw Data 
  f_ud = np.round(scipy.signal.savgol_filter(data_y[2], window_size, 2, mode='nearest'));

  # Step2 Take The Derivative Of Yaw data
  diff_ud = np.diff(f_ud)

  # Step3: Calulate noise floor 

  nonzero_ud = np.nonzero(diff_ud)
  diff_ud_val = diff_ud[nonzero_ud]
  diff_ud_mean = np.mean(diff_ud_val)
  diff_ud_std = np.std(diff_ud[nonzero_ud])

  # Step4: Retrieve Player Output
  input_y = np.zeros(len(diff_ud))
  for i in range(len(diff_ud)-1):
    if diff_ud[i] < -1* (diff_ud_mean+diff_ud_std*factor_noisefloor):
      input_y[i] = -1
    elif diff_ud[i] > (diff_ud_mean+diff_ud_std*factor_noisefloor):
      input_y[i] = 1
    else:
      input_y[i] = 0
  return input_y
